In [16]:
import pandas as pd
from cleaned_data import *
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from src.item_recommender import ItemRecommender
import re

In [5]:
data=pd.read_csv('movies_metadata.csv')
movies=pd.read_csv('cleaned_data/movies.csv')

In [228]:
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [81]:
kaggle=data[['id','title','overview','imdb_id']]

In [49]:
movies['movie_title'] = movies['movie_title'].apply(lambda x: x.split(' (')[0])

In [66]:

titles=[]
overview=[]
for i in data['original_title']:
    titles.append(i)
for j in data['overview']:
    overview.append(j)


In [68]:
dictionary = dict(zip(titles, overview))
dictionary

{'Toy Story': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'Jumanji': "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",
 'Grumpier Old Men': "A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. Bu

In [69]:
movies['description'] = movies['movie_title'].map(dictionary)

In [100]:
movies['description']=movies['description'].fillna(value='')

In [111]:
movies['bow'] = movies['genre']+' '+movies['description']

In [242]:
movies

,movie_id,movie_title,genre,year,description,bow
0,1,Toy Story,Animation Children's Comedy,1995,"Led by Woody, Andy's toys live happily in his ...","Animation Children's Comedy Led by Woody, Andy..."
1,2,Jumanji,Adventure Children's Fantasy,1995,When siblings Judy and Peter discover an encha...,Adventure Children's Fantasy When siblings Jud...
2,3,Grumpier Old Men,Comedy Romance,1995,A family wedding reignites the ancient feud be...,Comedy Romance A family wedding reignites the ...
3,4,Waiting to Exhale,Comedy Drama,1995,"Cheated on, mistreated and stepped on, the wom...","Comedy Drama Cheated on, mistreated and steppe..."
4,5,Father of the Bride Part II,Comedy,1995,Just when George Banks has recovered from his ...,Comedy Just when George Banks has recovered fr...
...,...,...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000,"Greg Focker is ready to marry his girlfriend, ...",Comedy Greg Focker is ready to marry his girlf...
3879,3949,Requiem for a Dream,Drama,2000,The hopes and dreams of four ambitious people ...,Drama The hopes and dreams of four ambitious p...
3880,3950,Tigerland,Drama,2000,A group of recruits go through Advanced Infant...,Drama A group of recruits go through Advanced ...
3881,3951,Two Family House,Drama,2000,Buddy Visalo (Michael Rispoli) is a factory wo...,Drama Buddy Visalo (Michael Rispoli) is a fact...


In [233]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
vectorizer.fit(movies['bow'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [234]:
description_matrix = vectorizer.transform(movies['description'])
genre_matrix = vectorizer.transform(movies['genre'])

In [235]:
# # instantiating and generating the count matrix
# count = CountVectorizer()
# count_matrix = count.fit_transform(movies['cv'])

In [236]:
def weighted_vector(d_matrix, g_matrix, alpha):
    return (d_matrix * alpha) + (g_matrix * (1-alpha))

In [237]:
weighted_vector2 = weighted_vector(description_matrix, genre_matrix, .4)

In [238]:
weighted_recommender = ItemRecommender()

In [240]:
weighted_recommender.fit(weighted_vector2, index = movies['movie_id'])
print(weighted_recommender.get_recommendations())

[2142 2141 2354 2355 3754]


In [217]:
cosine_sim = cosine_similarity(description_matrix)

In [230]:
sum(cosine_sim[-1])

0.0

In [ ]:
#add in if sum of cosine_sim ==0 then set alpha to 0

In [ ]:
#why does cosine_sim have a length of 3883 - the same amount of rows in the dataset? shouldnt it be equal to the set max features in the vectorizer?